In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from typing import Tuple, Union, Optional

from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    mean_absolute_percentage_error,
)
import matplotlib.pyplot as plt

In [2]:
# =========================
# 2. LOAD Processed DATA
# =========================
df = pd.read_csv("../data/processing/processed_v2.csv")
volatility = "volatility_rolling_28"
zscore = "zscore_rolling_28"

df["date"] = pd.to_datetime(df["date"])
df = df.sort_values("date").reset_index(drop=True)

In [3]:
# =========================
# 4. SUPERVISED DATASET
# =========================

features = ["log_returns", volatility, zscore]
target = "log_returns"

lookback = 21
train_ratio = 0.75


def make_sequences(df_feat, target_col, lookback):
    X, y = [], []

    for i in range(len(df_feat) - lookback):
        X.append(df_feat.iloc[i : i + lookback].values.flatten())
        y.append(df_feat.iloc[i + lookback][target_col])

    return np.array(X), np.array(y)


# ===== build sequences
data = df[features]
X, y = make_sequences(data, target, lookback)

split = int(len(X) * train_ratio)

X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]


# =========================
# EXPORT CSV
# =========================
col_names = [f"{f}_t-{lookback-t}" for t in range(lookback) for f in features]

df_train = pd.DataFrame(X_train, columns=col_names)
df_train["target"] = y_train

df_test = pd.DataFrame(X_test, columns=col_names)
df_test["target"] = y_test

df_train.to_csv("../data/datasets/train_sequences.csv", index=False)
df_test.to_csv("../data/datasets/test_sequences.csv", index=False)

In [4]:
X_train = X_train.reshape(-1, lookback, len(features))
X_test  = X_test.reshape(-1, lookback, len(features))

In [5]:
class StandardScalerCustom:
    """
    Class thực hiện chuẩn hóa Z-score (Standardization) cho dữ liệu.
    Hỗ trợ cả dữ liệu đặc trưng (X) và biến mục tiêu (y).
    """

    def __init__(self) -> None:
        self.mu: Optional[Union[np.ndarray, float]] = None
        self.std: Optional[Union[np.ndarray, float]] = None

    def fit(
        self, data: np.ndarray, axis: Optional[Union[int, Tuple[int, ...]]] = None
    ) -> None:
        """
        Tính toán giá trị trung bình và độ lệch chuẩn từ tập dữ liệu huấn luyện.

        Args:
            data: Mảng numpy chứa dữ liệu huấn luyện.
            axis: Trục để tính toán (ví dụ: (0, 1) cho dữ liệu ảnh/chuỗi).
        """
        # keepdims=True để đảm bảo tính toán broadcasting sau này không bị lỗi
        self.mu = (
            np.mean(data, axis=axis, keepdims=True)
            if axis is not None
            else np.mean(data)
        )
        self.std = (
            np.std(data, axis=axis, keepdims=True) if axis is not None else np.std(data)
        )

        # Thêm epsilon để tránh chia cho 0
        self.std += 1e-8

    def encode(self, data: np.ndarray) -> np.ndarray:
        """
        Chuẩn hóa dữ liệu (Transform/Encode).

        Args:
            data: Mảng dữ liệu cần chuẩn hóa.
        Returns:
            Dữ liệu đã chuẩn hóa có cùng shape với đầu vào.
        """
        if self.mu is None or self.std is None:
            raise ValueError("Bạn phải gọi hàm .fit() trước khi encode!")

        return (data - self.mu) / self.std

    def decode(self, normalized_data: np.ndarray) -> np.ndarray:
        """
        Giải chuẩn hóa dữ liệu về đơn vị gốc (Inverse Transform/Decode).
        Thường dùng cho y_pred để xem kết quả thực tế.

        Args:
            normalized_data: Dữ liệu đã chuẩn hóa.
        """
        if self.mu is None or self.std is None:
            raise ValueError("Bạn phải gọi hàm .fit() trước khi decode!")

        return (normalized_data * self.std) + self.mu

In [6]:
# Khởi tạo scaler cho X
scaler_X = StandardScalerCustom()

# Bước 1: Fit - Tính X_mu và X_std từ X_train
# Tương đương: X_mu = X_train.mean(axis=(0, 1), keepdims=True)
scaler_X.fit(X_train, axis=(0, 1))

# Bước 2: Encode - Chuẩn hóa X_train và X_test bằng X_mu, X_std vừa tính
# Tương đương: X_train = (X_train - X_mu) / X_std
X_train = scaler_X.encode(X_train)
X_test = scaler_X.encode(X_test)


# Khởi tạo scaler cho y
scaler_y = StandardScalerCustom()

# Bước 1: Fit - Tính y_mu và y_std từ y_train
# Tương đương: y_mu = y_train.mean()
scaler_y.fit(y_train)

# Bước 2: Encode - Chuẩn hóa y_train và y_test
y_train = scaler_y.encode(y_train)
y_test = scaler_y.encode(y_test)

In [7]:
# =========================
# 6. DATASET + DATALOADER
# =========================
class ReturnDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X).float()
        self.y = torch.tensor(y).float().unsqueeze(-1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


train_loader = DataLoader(ReturnDataset(X_train, y_train), batch_size=32, shuffle=False)

test_loader = DataLoader(ReturnDataset(X_test, y_test), batch_size=32, shuffle=False)


In [8]:
class LSTMRegressor(nn.Module):
    def __init__(
        self,
        input_dim: int = len(features),
        hidden_dim: int = 32,
        num_layers: int = 5,
    ):
        super().__init__()

        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
        )

        # chỉ 1 linear layer
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out, _ = self.lstm(x)

        # lấy timestep cuối
        last_hidden = out[:, -1, :]

        out = self.fc(last_hidden)

        return out


model = LSTMRegressor(input_dim=len(features))
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)

In [9]:
log_dir = f"runs/lstm_regressor_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
writer = SummaryWriter(log_dir)

In [10]:
epochs = 300

for epoch in range(epochs):
    model.train()
    train_loss_sum = 0.0

    for xb, yb in train_loader:
        optimizer.zero_grad()

        preds = model(xb)
        loss = criterion(preds, yb)

        loss.backward()

        # vẫn giữ clip để tránh exploding gradient (nên giữ cái này)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)

        optimizer.step()

        train_loss_sum += loss.item() * len(xb)

    train_loss = train_loss_sum / len(train_loader.dataset)

    print(f"Epoch {epoch+1:3d} | Loss {train_loss:.6f}")

    # ===== epoch log =====
    writer.add_scalar("train/loss", train_loss, epoch)

    print(f"Epoch {epoch+1:3d} | Loss {train_loss:.6f}")

    # ===== optional: weight histogram =====
    for name, param in model.named_parameters():
        writer.add_histogram(name, param, epoch)

Epoch   1 | Loss 1.003706
Epoch   1 | Loss 1.003706
Epoch   2 | Loss 1.001413
Epoch   2 | Loss 1.001413
Epoch   3 | Loss 1.001148
Epoch   3 | Loss 1.001148
Epoch   4 | Loss 1.001032
Epoch   4 | Loss 1.001032
Epoch   5 | Loss 1.000944
Epoch   5 | Loss 1.000944
Epoch   6 | Loss 1.000849
Epoch   6 | Loss 1.000849
Epoch   7 | Loss 1.000573
Epoch   7 | Loss 1.000573
Epoch   8 | Loss 0.998920
Epoch   8 | Loss 0.998920
Epoch   9 | Loss 1.001004
Epoch   9 | Loss 1.001004
Epoch  10 | Loss 1.000833
Epoch  10 | Loss 1.000833
Epoch  11 | Loss 1.000716
Epoch  11 | Loss 1.000716
Epoch  12 | Loss 1.000649
Epoch  12 | Loss 1.000649
Epoch  13 | Loss 1.000547
Epoch  13 | Loss 1.000547
Epoch  14 | Loss 1.000372
Epoch  14 | Loss 1.000372
Epoch  15 | Loss 0.997907
Epoch  15 | Loss 0.997907
Epoch  16 | Loss 0.999872
Epoch  16 | Loss 0.999872
Epoch  17 | Loss 0.996791
Epoch  17 | Loss 0.996791
Epoch  18 | Loss 0.997090
Epoch  18 | Loss 0.997090
Epoch  19 | Loss 0.993073
Epoch  19 | Loss 0.993073
Epoch  20 | 

In [11]:
# torch.save(model.state_dict(), "lstm_model.pt")

In [12]:
def predict(loader):
    preds, trues = [], []

    model.eval()
    with torch.no_grad():
        for xb, yb in loader:
            out = model(xb)

            preds.append(out.squeeze(-1).cpu().numpy())
            trues.append(yb.squeeze(-1).cpu().numpy())

    return np.concatenate(preds), np.concatenate(trues)

def evaluate_and_log(name, y_pred, y_true, epoch):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    writer.add_scalar(f"{name}/mse", mse, epoch)
    writer.add_scalar(f"{name}/rmse", rmse, epoch)
    writer.add_scalar(f"{name}/mae", mae, epoch)
    writer.add_scalar(f"{name}/r2", r2, epoch)

    print(
        f"{name.upper()} | "
        f"MSE {mse:.6f} | RMSE {rmse:.6f} | MAE {mae:.6f} | R2 {r2:.4f}"
    )

    fig = plt.figure(figsize=(6, 6))
    plt.scatter(y_true, y_pred, s=5, alpha=0.5)
    plt.plot([y_true.min(), y_true.max()],
             [y_true.min(), y_true.max()],
             linestyle="--")
    writer.add_figure(f"{name}/pred_vs_true", fig, epoch)
    plt.close()


In [13]:
def recursive_forecast_log_return(
    model,
    df: pd.DataFrame,
    start_index,
    n_steps,
    lookback,
    scaler_X,
    scaler_y,
    features,
    target,
    rolling_window=28,
    volatility_col="volatility_rolling_28",
    zscore_col="zscore_rolling_28"
):
    model.eval()

    # 1. Lấy dữ liệu lịch sử ban đầu
    window_data = df.loc[start_index - lookback : start_index - 1].copy()
    
    # Tạo window với đúng các features
    window = window_data[features].values.copy()
    
    logret_preds = []
    
    # Tìm indices cho các features
    target_idx = features.index(target) if target in features else 0
    vol_idx = features.index(volatility_col) if volatility_col in features else None
    z_idx = features.index(zscore_col) if zscore_col in features else None
    
    # 2. Chuẩn bị rolling statistics cho các bước đầu tiên
    # Lấy lịch sử log_returns thực tế
    ret_hist = df["log_returns"].iloc[max(0, start_index - rolling_window):start_index].tolist()
    
    # 3. Vòng lặp dự đoán
    for step in range(n_steps):
        # A. Chuẩn hóa và dự đoán
        # Đảm bảo window có shape (1, lookback, n_features)
        window_scaled = scaler_X.encode(window.reshape(1, lookback, len(features)))
        x = torch.tensor(window_scaled).float()
        
        with torch.no_grad():
            y_hat_norm = model(x).item()
        
        # B. Giải chuẩn hóa để có log_return thực
        new_log_ret = float(scaler_y.decode(y_hat_norm))
        logret_preds.append(new_log_ret)
        
        # C. Cập nhật rolling statistics
        ret_hist.append(new_log_ret)
        if len(ret_hist) > rolling_window:
            ret_hist = ret_hist[-rolling_window:]
        
        mu_rolling = np.mean(ret_hist)
        sigma_rolling = np.std(ret_hist) + 1e-8
        
        # D. Tính các features mới
        new_vol = sigma_rolling
        new_z = (new_log_ret - mu_rolling) / sigma_rolling if sigma_rolling != 0 else 0
        
        # E. Tạo hàng mới với tất cả features
        new_row = np.zeros(len(features))
        new_row[target_idx] = new_log_ret
        
        if vol_idx is not None:
            new_row[vol_idx] = new_vol
        if z_idx is not None:
            new_row[z_idx] = new_z
        
        # Nếu có các features khác (như ema_dist), giữ giá trị cũ hoặc tính toán lại
        for i in range(len(features)):
            if i not in [target_idx, vol_idx, z_idx]:
                # Giữ giá trị từ timestep trước (hoặc tính toán lại nếu cần)
                new_row[i] = window[-1, i]
        
        # F. Cập nhật window: bỏ hàng đầu, thêm hàng mới
        window = np.vstack([window[1:], new_row])
    
    return np.array(logret_preds)

In [14]:
import plotly.graph_objects as go
import numpy as np

# ===== 1. Predict trên tập TRAIN =====
y_train_pred_norm, y_train_true_norm = predict(train_loader)

# ===== 2. Inverse transform (Dùng lại scaler_y đã fit) =====
y_train_pred = scaler_y.decode(y_train_pred_norm)
y_train_true = scaler_y.decode(y_train_true_norm)


# ===== 3. Map dates cho tập TRAIN =====
# Logic cũ: Test lấy từ split trở đi ([split:])
# Logic mới: Train lấy từ đầu đến split ([:split])
all_target_idx = np.arange(lookback, len(df))
train_target_idx = all_target_idx[:split] 
dates_train = df["date"].iloc[train_target_idx].reset_index(drop=True)


# =========================
# ===== 4. Vẽ biểu đồ =====
# =========================
fig = go.Figure()

# Actual Train
fig.add_trace(go.Scatter(
    x=dates_train,
    y=y_train_true.flatten(), 
    mode="lines",
    name="Actual (Train)",
    line=dict(width=1.5, color="blue") # Đổi màu xíu cho dễ phân biệt
))

# Predicted Train
fig.add_trace(go.Scatter(
    x=dates_train,
    y=y_train_pred.flatten(),
    mode="lines",
    name="Predicted (Train)",
    opacity=0.8,
    line=dict(color="orange")
))

# Zero line
fig.add_hline(y=0, line_width=1, line_dash="dash")

# ===== Layout tuning =====
fig.update_layout(
    title="Log Returns — Training Set (Actual vs Pred)",
    xaxis_title="Date",
    yaxis_title="log_returns",
    hovermode="x unified",
    height=450,
    legend=dict(orientation="h", y=1.05)
)

fig.show()

In [15]:
import plotly.graph_objects as go
import numpy as np

# ===== predict =====
# Giả định predict trả về numpy array
y_pred_norm, y_true_norm = predict(test_loader)

# ===== THAY ĐỔI: Inverse transform bằng Class =====
# Thay vì: y_pred = y_pred_norm * y_std + y_mu
# Ta dùng:
y_pred = scaler_y.decode(y_pred_norm)
y_true = scaler_y.decode(y_true_norm)

# Nếu predict trả về list hoặc tensor, đảm bảo convert sang numpy trước khi decode
# y_pred = scaler_y.decode(np.array(y_pred_norm)) 


# ===== map dates (Giữ nguyên) =====
all_target_idx = np.arange(lookback, len(df))
test_target_idx = all_target_idx[split:]
dates_test = df["date"].iloc[test_target_idx].reset_index(drop=True)


# =========================
# =========================
fig = go.Figure()

# actual
fig.add_trace(go.Scatter(
    x=dates_test,
    y=y_true.flatten(), # flatten() để đảm bảo mảng là 1D cho plotly vẽ đẹp
    mode="lines",
    name="Actual log_returns",
    line=dict(width=1.5)
))

# predicted
fig.add_trace(go.Scatter(
    x=dates_test,
    y=y_pred.flatten(), # flatten() tương tự
    mode="lines",
    name="Predicted log_returns",
    opacity=0.8
))

# zero line
fig.add_hline(y=0, line_width=1, line_dash="dash")


# ===== layout tuning =====
fig.update_layout(
    title="Log Returns — Test Set (Actual vs Pred)",
    xaxis_title="Date",
    yaxis_title="log_returns",
    hovermode="x unified",
    height=450,
    legend=dict(orientation="h", y=1.05)
)

fig.show()

In [16]:
START_INDEX = split + lookback  # điểm bắt đầu dự đoán
N_STEPS = 430  # số ngày dự đoán tiếp

logret_pred = recursive_forecast_log_return(
    model=model,
    df=df,
    start_index=START_INDEX,
    n_steps=N_STEPS,
    lookback=lookback,
    scaler_X=scaler_X,  # Truyền object scaler_X
    scaler_y=scaler_y,  # Truyền object scaler_y
    features=features,
    target=target,
    volatility_col=volatility,  # THÊM THAM SỐ NÀY
    zscore_col=zscore  # THÊM THAM SỐ NÀY
)

anchor_close = df["close"].iloc[START_INDEX - 1]

close_pred = anchor_close * np.exp(np.cumsum(logret_pred))

df_rec = df.iloc[START_INDEX : START_INDEX + N_STEPS].copy()
df_rec["close_pred"] = close_pred


fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df_rec["date"],
        y=df_rec["close"],
        name="Actual Close",
        line=dict(color="black"),
    )
)

fig.add_trace(
    go.Scatter(
        x=df_rec["date"],
        y=df_rec["close_pred"],
        name="Recursive Prediction",
        line=dict(color="red"),
    )
)


start_date = df_rec["date"].iloc[0]

fig.add_shape(
    type="line",
    x0=start_date,
    x1=start_date,
    y0=0,
    y1=1,
    xref="x",
    yref="paper",
    line=dict(color="blue", dash="dash"),
)

fig.add_annotation(
    x=start_date,
    y=1,
    yref="paper",
    text="Prediction Start",
    showarrow=False,
    xanchor="left",
    yanchor="bottom",
)

fig.update_layout(
    title="Recursive Forecast (log_return → close)",
    xaxis_title="Time",
    yaxis_title="Price",
    legend=dict(x=0.01, y=0.99),
)

fig.show()

In [17]:
mape_train = mean_absolute_percentage_error(
    y_train_true.flatten(), y_train_pred.flatten()
)

# Tính cho tập Test
mape_test = mean_absolute_percentage_error(y_true.flatten(), y_pred.flatten())

print(f"Train MAPE: {mape_train * 100:.2f}%")
print(f"Test MAPE: {mape_test * 100:.2f}%")

Train MAPE: 148.81%
Test MAPE: 1335.47%


In [18]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# ==========================================
# 1. CẤU HÌNH CHO TẬP TRAIN
# ==========================================

# Điểm bắt đầu: Ngay sau khoảng lookback đầu tiên
ROLLING_WINDOW = 28
TRAIN_START_INDEX = max(lookback, ROLLING_WINDOW)

# Số bước: Từ điểm bắt đầu đến điểm chia tập (split)
# Lưu ý: Nếu tập train quá lớn, bạn có thể giảm số này xuống (ví dụ 200) để test nhanh
TRAIN_N_STEPS = 400

print(f"Đang chạy Recursive Forecast trên tập Train từ index {TRAIN_START_INDEX} đến {TRAIN_START_INDEX + TRAIN_N_STEPS}...")

# ==========================================
# 2. THỰC HIỆN DỰ BÁO
# ==========================================
logret_train_pred = recursive_forecast_log_return(
    model=model,
    df=df,
    start_index=TRAIN_START_INDEX,
    rolling_window=ROLLING_WINDOW,
    n_steps=TRAIN_N_STEPS,
    lookback=lookback,
    scaler_X=scaler_X,
    scaler_y=scaler_y,
    features=features,
    target=target,
)

# ==========================================
# 3. TÁI TẠO GIÁ CLOSE (RECONSTRUCT)
# ==========================================

# Lấy giá đóng cửa tại thời điểm ngay trước khi bắt đầu dự báo để làm mốc (Anchor)
anchor_close_train = df["close"].iloc[TRAIN_START_INDEX - 1]

# Tính giá Close dự đoán từ log_returns dự đoán
# Công thức: Price_t = Price_0 * exp(cumsum(log_ret))
close_train_pred = anchor_close_train * np.exp(np.cumsum(logret_train_pred))

# Tạo DataFrame tạm để đối chiếu
df_train_rec = df.iloc[TRAIN_START_INDEX : TRAIN_START_INDEX + TRAIN_N_STEPS].copy()
df_train_rec["close_pred"] = close_train_pred

# ==========================================
# 4. VẼ BIỂU ĐỒ (PLOTLY)
# ==========================================

fig = go.Figure()

# 1. Giá thực tế (Actual Close)
fig.add_trace(go.Scatter(
    x=df_train_rec["date"],
    y=df_train_rec["close"],
    mode="lines",
    name="Actual Close (Train)",
    line=dict(color="blue", width=1.5)
))

# 2. Giá dự báo Recursive (Predicted Close)
fig.add_trace(go.Scatter(
    x=df_train_rec["date"],
    y=df_train_rec["close_pred"],
    mode="lines",
    name="Recursive Forecast (Train)",
    line=dict(color="orange", width=1.5),
    opacity=0.9
))

fig.update_layout(
    title=f"Backtest on Training Data ({TRAIN_N_STEPS} steps)",
    xaxis_title="Date",
    yaxis_title="Price (Close)",
    hovermode="x unified",
    height=500,
    template="plotly_white"
)

fig.show()

Đang chạy Recursive Forecast trên tập Train từ index 28 đến 428...
